#Twitter Sentiment Analysis

Aranged by StudentID-StudentName:

12S18033-Cristina Sriwahyuni Hasibuan

12S18038-Naomi A. Simatupang

12S18049-Natasya Sitorus

12S18060-Elsaday Bakara

##Requirements installation if not exist yet

In [1]:
pip install pyspark

In [4]:
!pip install tweet-preprocessor

##Import Lib

In [2]:
import pyspark
import json
import csv
import nltk
from nltk.tokenize import word_tokenize
import re
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import tweepy
from tweepy import OAuthHandler
from tweepy import Stream
from collections import Counter
from plotly import graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff
from pyspark.sql.types import StringType
from pyspark.sql.functions import col, udf
from pyspark.ml.feature import Tokenizer, StringIndexer, Word2Vec, StopWordsRemover, HashingTF
from pyspark.ml import Pipeline, Transformer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics

In [5]:
from wordcloud import WordCloud, STOPWORDS
from datetime import timedelta, datetime
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import matplotlib.pyplot as plt 
from tweepy import OAuthHandler
from textblob import TextBlob
import preprocessor as p
import seaborn as sns
import pandas as pd
import numpy as np
import tweepy
import csv
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

##Configuration

In [20]:
APIKey = "XEu6VXcA06Aneo8y0JYiZoC34"
APISecretKey = "lKRAifMEA6GOIPRzsGDLQ4RKNBUT9EfAjTFRiztmVzdFBmRwln"
AccessToken = "1390698546353999878-65J1HL6hFuH2BSZyMsDOn5xJiIIl1C"
AccessTokenSecret = "HvMZfhKfXQv0NFiWRmdXRl27cYfsHpQsImw0okSuyQpvg"

In [9]:
# Quick Observation based on twitter search box by number

# Quite Few (fewer than 50):
# 'testimoni scarlett whitening'
# 'scarlett whitening testimoni'

# Pretty Much:
# 'scarlett whitening review'
# 'review scarlett whitening'
# 'scarlett whitening'

# Not reccomended :
# 'scarlett review', Bcz the result is irrelevant
search_words = ['scarlett whitening review']

In [18]:
today = datetime.today().strftime("%Y-%m-%d")
last_week = datetime.today() - timedelta(7)
last_week = last_week.strftime("%Y-%m-%d")

search_words = ['#scarlettwhiteningreview OR #reviewscarlettwhitening OR #scarlettwhitening']
date_since = last_week
date_until = today

##Set Up

In [10]:
OAuthKeys = {'consumer_key':APIKey, 'consumer_secret':APISecretKey, 'access_token_key':AccessToken, 'access_token_secret':AccessTokenSecret}

In [11]:
auth = tweepy.OAuthHandler(OAuthKeys['consumer_key'], OAuthKeys['consumer_secret'])

In [12]:
API = tweepy.API(auth)

In [13]:
def scraptweets(search_words, date_since, date_until):

    db_tweets = pd.DataFrame(columns=['username', 'tweetcreatedts', 'text'])

    tweets = tweepy.Cursor(
                    API.search, q=search_words, lang="id", 
                    since=date_since, until=date_until,  tweet_mode='extended').items(1000)

    tweet_list = [tweet for tweet in tweets]

    for tweet in tweet_list:
        username = tweet.user.screen_name
        tweetcreatedts = tweet.created_at

        try:
            text = tweet.retweeted_status.full_text
        except AttributeError:
            text = tweet.full_text

        ith_tweet = [username, tweetcreatedts, text]

        db_tweets.loc[len(db_tweets)] = ith_tweet
    
    print('Proses Scrapping Selesai Dengan Jumlah Data', len(db_tweets))
    filename = 'reviews.csv'
    db_tweets.to_csv(filename, index=False)

In [19]:
scraptweets(search_words, date_since, date_until)

TweepError: ignored

In [ ]:
def toDataFrame(tweets):
  data = pd.DataFrame
  data.['tweetID'] = [tweet.id for tweet in tweets]
  data.['tweetText'] = [tweet.text.encode('utf-8') for tweet in tweets]
  data.['tweetCreated'] = [tweet.created_at for tweet in tweets]
  data.['userID'] = [tweet.user.id for tweet in tweets]
  data.['username'] = [tweet.user.name for tweet in tweets]
  return data

data = toDataFrame(results)
data.to_csv('reviews.csv', index=False)

##Clean Tweets

In [ ]:
def clean_tweet(self, tweet): 
    ''' 
    Use sumple regex statemnents to clean tweet text by removing links and special characters
    '''
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t]) \
                                |(\w+:\/\/\S+)", " ", tweet).split()) 
def deEmojify(text):
    '''
    Strip all non-ASCII characters to remove emoji characters
    '''
    if text:
        return text.encode('ascii', 'ignore').decode('ascii')
    else:
        return None

In [ ]:
data['tweetText'].clean_tweet
data['tweetText'].deEmojify

##Spark Session

In [ ]:
#memulai Spark Session
from pyspark.sql import SparkSession
spark = SparkSession.builder \
  .master("local[2]") \
  .appName("Proyek_Sentiment Analysis") \
  .getOrCreate()


In [ ]:
# melakukan load dataset menggunakan library pandas (dikarenakan Spark load data tidak sesuai)
clean_data = data
# melakukan load dataset kedalam Spark dan menampilkannya
reviews = spark.createDataFrame(clean_data)
reviews.show(10)
# melakukan Konversi Data ke format Parquet
reviews.write.format("parquet").mode("overwrite").save("D:/semester8/PDB/Proyek/Data_Parquet")
